In [3]:
from PIL import Image
from models import Creature, ToolWeapon
from pathlib import Path

import json
import base64
import os
import io

In [4]:
creatures_json = json.load(open('./data/crawled/1.4.1.4512/bestiary.json', 'r', encoding='utf-8'))
creatures: list[Creature] = []
for creature_key, creature_data in creatures_json.items():
    creature = Creature.from_dict(creature_data)
    creatures.append(creature)

In [5]:
def interpolate_color(start: tuple, end: tuple, minimum: float, maximum: float, value: float) -> tuple:
    start_r, start_g, start_b = start
    end_r, end_g, end_b = end
    r = start_r + (end_r - start_r) * ((value - minimum) / (maximum - minimum))
    g = start_g + (end_g - start_g) * ((value - minimum) / (maximum - minimum))
    b = start_b + (end_b - start_b) * ((value - minimum) / (maximum - minimum))
    return (int(r), int(g), int(b))

In [6]:
def build_resistance_weakness(damage_type: str, creature: Creature) -> str:
    value = creature.get_weakness_or_resistance(damage_type)
    value = value - 1
    color = None
    if value < 0:
        color = interpolate_color(
            start=(255, 0, 0),
            end=(255, 255, 255),
            minimum=-1,
            maximum=0,
            value=value
        )
    elif value > 0:
        color = interpolate_color(
            start=(255, 255, 255),
            end=(0, 255, 0),
            minimum=0,
            maximum=1,
            value=value
        )
    
    value *= 100
    if value == 0:
        return '<td>-</td>'
    return f'<td style="background-color: rgb{color}; font-weight: bold;">{value:.0f} %</td>'

In [7]:
creatures_table = '<table class="table table-striped table-bordered table-hover table-condensed">'
creatures_table += '''<thead>
    <tr>
        <th>Name</th>
        <th>Health</th>
        <th>Generic</th>
        <th>Busting</th>
        <th>Chopping</th>
        <th>Slashing</th>
        <th>Stabbing</th>
        <th>Fresh</th>
        <th>Salty</th>
        <th>Sour</th>
        <th>Spicy</th>
        <th>Burning</th>
        <th>Explosive</th>
        <th>Shock</th>
    </tr>
</thead>
'''
creatures_table += '<tbody>'
for creature in creatures:
    creatures_table += '<tr>'
    if creature.name is not None:
        creatures_table += f'<td>{creature.name.text}</td>'
    else:
        creatures_table += f'<td>{creature.key_name}</td>'
        
    if creature.info is None:
        creatures_table += '<td>Unknown</td>'
    else:
        creatures_table += f'<td>{creature.info.health}</td>'

    creatures_table += f'{build_resistance_weakness("general", creature)}'
    creatures_table += f'{build_resistance_weakness("smashing", creature)}'
    creatures_table += f'{build_resistance_weakness("chopping", creature)}'
    creatures_table += f'{build_resistance_weakness("slashing", creature)}'
    creatures_table += f'{build_resistance_weakness("stabbing", creature)}'
    creatures_table += f'{build_resistance_weakness("fresh", creature)}'
    creatures_table += f'{build_resistance_weakness("salty", creature)}'
    creatures_table += f'{build_resistance_weakness("sour", creature)}'
    creatures_table += f'{build_resistance_weakness("spicy", creature)}'
    creatures_table += f'{build_resistance_weakness("burning", creature)}'
    creatures_table += f'{build_resistance_weakness("explosive", creature)}'
    creatures_table += f'{build_resistance_weakness("shock", creature)}'

    creatures_table += '</tr>'
creatures_table += '</tbody>'
creatures_table += '</table>'

html = f'''
<!DOCTYPE html>
<html>
<head>
    <title>Bestiary</title>
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css">
</head>
<body>
    <h1>Bestiary</h1>
    <!-- filters panel -->
    <div class="panel panel-default">
        <div class="panel-heading">Filters</div>
        <div class="panel-body">
            <div class="form-group">
                <label for="search">Search</label>
                <input type="text" id="search" class="form-control" placeholder="Search...">
            </div>
        </div>
    </div>
    <!-- bestiary table -->
    {creatures_table}
</body>
<script>
document.addEventListener('DOMContentLoaded', () => {{
    // Add Hover Effect to Table Rows
    const table = document.querySelector('table');
    table.addEventListener('mouseover', (event) => {{
        if (event.target.tagName === 'TD') {{
            const row = event.target.parentElement;
            row.style.backgroundColor = '#f5f5f5';
            let cells = row.getElementsByTagName('td');
            for (let i = 0; i < cells.length; i++) {{
                let originalColor = cells[i].style.backgroundColor;
                let r, g, b;
                if (originalColor === '') {{
                    r = 255;
                    g = 255;
                    b = 255;
                }} else {{
                    let rgb = originalColor.match(/rgb\\((\\d+), (\\d+), (\\d+)\\)/);
                    r = parseInt(rgb[1]);
                    g = parseInt(rgb[2]);
                    b = parseInt(rgb[3]);
                }}
                cells[i].style.backgroundColor = `rgb(${{r + 20}}, ${{g + 20}}, ${{b + 20}})`;
                
                cells[i].setAttribute('data-original-color', originalColor);
            }}
        }}
    }});
    table.addEventListener('mouseout', (event) => {{
        if (event.target.tagName === 'TD') {{
            const row = event.target.parentElement;
            row.style.backgroundColor = '';
            let cells = row.getElementsByTagName('td');
            for (let i = 0; i < cells.length; i++) {{
                let originalColor = cells[i].getAttribute('data-original-color');
                cells[i].style.backgroundColor = originalColor;
            }}
        }}
    }});
}});

document.addEventListener('DOMContentLoaded', () => {{
    const tableHeaders = document.querySelectorAll('th');
    tableHeaders.forEach((header, index) => {{
        header.addEventListener('click', () => {{
            const table = header.closest('table');
            const tbody = table.querySelector('tbody');
            const rows = Array.from(tbody.querySelectorAll('tr'));
            const isAscending = header.classList.contains('asc');

            rows.sort((rowA, rowB) => {{
                const textA = rowA.querySelectorAll('td')[index].textContent.trim().toLowerCase();
                const textB = rowB.querySelectorAll('td')[index].textContent.trim().toLowerCase();

                // Converte "-" para "0" e extrai números de strings (incluindo porcentagens)
                const cellA = textA === '-' ? 0 : (textA.endsWith('%') ? parseFloat(textA.replace('%', '')) : parseFloat(textA));
                const cellB = textB === '-' ? 0 : (textB.endsWith('%') ? parseFloat(textB.replace('%', '')) : parseFloat(textB));

                if (!isNaN(cellA) && !isNaN(cellB)) {{ // if both cells contain numeric values
                    return isAscending ? cellA - cellB : cellB - cellA;
                }} else {{ // if cells contain non-numeric text
                    return isAscending ? textA.localeCompare(textB) : textB.localeCompare(textA);
                }}
            }});

            // Toggle sort direction for next click
            header.classList.toggle('asc', !isAscending);
            header.classList.toggle('desc', isAscending);

            // Append sorted rows back to the tbody
            rows.forEach(row => tbody.appendChild(row));
        }});
    }});
}});

document.addEventListener('DOMContentLoaded', () => {{
    const searchInput = document.getElementById('search');
    searchInput.addEventListener('input', () => {{
        const query = searchInput.value.toLowerCase();
        const table = searchInput.closest('body').querySelector('table');
        const rows = Array.from(table.querySelectorAll('tbody tr'));

        rows.forEach(row => {{
            const cells = Array.from(row.querySelectorAll('td'));
            const rowText = cells.map(cell => cell.textContent.trim().toLowerCase()).join(' ');
            row.style.display = rowText.includes(query) ? '' : 'none';
        }});
    }});
}});
</script>
</html>
'''

with open('./data/crawled/1.4/bestiary.html', 'w', encoding='utf-8') as f:
    f.write(html)

In [9]:
weapons_json = json.load(open('./data/crawled/1.4.1.4512/tools_weapons.json', 'r', encoding='utf-8'))
weapons: list[ToolWeapon] = []
for weapon_key, weapon_data in weapons_json.items():
    weapon = ToolWeapon.from_dict(weapon_data)
    weapons.append(weapon)

In [10]:
damage_types = {
    'BP_ExplosiveDamage': ['explosive'],
    'BP_ChoppingDamage': ['chopping'],
    'BP_MiningDamage': ['busting'],
    'BP_StabbingDamage': ['stabbing'],
    'BP_GeneralDamage': ['generic'],
    'BP_FreshDamage': ['fresh'],
    'BP_SaltyDamage': ['salty'],
    'BP_SourDamage': ['sour'],
    'BP_SpicyDamage': ['spicy'],
    'BP_RepairDamage': ['repair'],
    'BP_DiggingDamage': ['digging'],
    'BP_SpicySlashingDamage': ['spicy', 'slashing'],
    'BP_FreshSlashingDamage': ['fresh', 'slashing'],
    'BP_SourSlashingDamage': ['sour', 'slashing'],
    'BP_SlashingDamage': ['slashing'],
    'BP_AcidDamage': ['acid'],
    'BP_ChoppingSlashingDamage': ['chopping', 'slashing'],
    'BP_SourChoppingDamage': ['sour', 'chopping'],
    'BP_SpicyChoppingDamage': ['spicy', 'chopping'],
    'BP_FreshChoppingDamage': ['fresh', 'chopping'],
    'BP_BurningDamage': ['burning'],
    'BP_SaltySmashingDamage': ['salty', 'smashing'],
}

In [11]:
def calculate_average_damage(weapon: ToolWeapon) -> float:
    if len(weapon.melee_attacks_info.main_combo) == 0:
        return 0

    damage = 0
    for index, attack in enumerate(weapon.melee_attacks_info.main_combo):
        damage += attack.main_damage_data.damage * weapon.melee_attacks_info.main_scaling[index]
    return damage / len(weapon.melee_attacks_info.main_combo)

def get_damage_type(weapon: ToolWeapon) -> str:
    if len(weapon.melee_attacks_info.main_combo) == 0:
        return 'None'

    damage_type_frequency = {}
    for attack in weapon.melee_attacks_info.main_combo:
        damage_type = attack.main_damage_data.damage_type
        if damage_type not in damage_type_frequency:
            damage_type_frequency[damage_type] = 0
        damage_type_frequency[damage_type] += 1

    return max(damage_type_frequency, key=damage_type_frequency.get)

def is_of_type(weapon: ToolWeapon, weapon_type: str) -> bool:
    weapon_damage_type = damage_types[get_damage_type(weapon)]
    return weapon_type in weapon_damage_type

def build_dmg_type_cell(weapon: ToolWeapon, weapon_type: str) -> str:
    has_type = is_of_type(weapon, weapon_type)
    color = 'rgb(255, 127, 127)' if not has_type else 'rgb(127, 255, 127)'
    text = 'Yes' if has_type else 'No'
    return f'<td style="background-color: {color}; font-weight: bold;">{text}</td>'

In [12]:
def get_icon_image(icon_path: Path, modicon_path: Path, size: int = 32) -> Image:
    icon = None
    if os.path.exists(icon_path) and os.path.isfile(icon_path):
        icon = Image.open(icon_path)
        if icon.mode != 'RGBA':
            icon = icon.convert('RGBA')
        icon = icon.resize((size, size), Image.LANCZOS)

        if os.path.exists(modicon_path) and os.path.isfile(modicon_path):
            modifier = Image.open(modicon_path)
            if modifier.mode != 'RGBA':
                modifier = modifier.convert('RGBA')
            modifier = modifier.resize((42, 42))

            try:
                icon.paste(modifier, (0, 0), modifier)
            except:
                return icon

    return icon

def image_to_base64(image: Image) -> str:
    if image is None:
        return ''
    buffer = io.BytesIO()
    image.save(buffer, format='PNG')
    return base64.b64encode(buffer.getvalue()).decode()

In [13]:
weapons_table = '<table class="table table-striped table-bordered table-hover table-condensed">'
weapons_table += '''<thead>
    <tr>
        <th>Image</th>
        <th>Name</th>
        <th>Average Damage</th>
        <th>Generic</th>
        <th>Busting</th>
        <th>Chopping</th>
        <th>Slashing</th>
        <th>Stabbing</th>
        <th>Fresh</th>
        <th>Salty</th>
        <th>Sour</th>
        <th>Spicy</th>
        <th>Burning</th>
        <th>Explosive</th>
        <th>Shock</th>
        <th>Damage Types</th>
    </tr>
</thead>
'''
weapons_table += '<tbody>'
for weapon in weapons:
    if len(weapon.melee_attacks_info.main_combo) == 0:
        continue

    icon = get_icon_image(weapon.icon_path, weapon.icon_modifier_path, size=64)
    base64_icon = image_to_base64(icon)
        
    weapons_table += '<tr>'
    weapons_table += f'<td><img src="data:image/png;base64,{base64_icon}" style="max-width: 100px; max-height: 100px;"></td>'
    weapons_table += f'<td>{weapon.display_name.text}</td>'
    weapons_table += f'<td>{calculate_average_damage(weapon):.2f}</td>'
    weapons_table += f'{build_dmg_type_cell(weapon, 'generic')}'
    weapons_table += f'{build_dmg_type_cell(weapon, 'busting')}'
    weapons_table += f'{build_dmg_type_cell(weapon, 'chopping')}'
    weapons_table += f'{build_dmg_type_cell(weapon, 'slashing')}'
    weapons_table += f'{build_dmg_type_cell(weapon, 'stabbing')}'
    weapons_table += f'{build_dmg_type_cell(weapon, 'fresh')}'
    weapons_table += f'{build_dmg_type_cell(weapon, 'salty')}'
    weapons_table += f'{build_dmg_type_cell(weapon, 'sour')}'
    weapons_table += f'{build_dmg_type_cell(weapon, 'spicy')}'
    weapons_table += f'{build_dmg_type_cell(weapon, 'burning')}'
    weapons_table += f'{build_dmg_type_cell(weapon, 'explosive')}'
    weapons_table += f'{build_dmg_type_cell(weapon, 'shock')}'
    weapons_table += f'<td>{', '.join(damage_types[get_damage_type(weapon)])}</td>'
    weapons_table += '</tr>'
weapons_table += '</tbody>'
weapons_table += '</table>'
html = f'''
<!DOCTYPE html>
<html>
<head>
    <title>Weapons</title>
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css">
</head>
<body>
    <h1>Weapons</h1>
    {weapons_table}
</body>
<script>
document.addEventListener('DOMContentLoaded', () => {{
    // Add Hover Effect to Table Rows
    const table = document.querySelector('table');
    table.addEventListener('mouseover', (event) => {{
        if (event.target.tagName === 'TD') {{
            const row = event.target.parentElement;
            row.style.backgroundColor = '#f5f5f5';
            let cells = row.getElementsByTagName('td');
            for (let i = 0; i < cells.length; i++) {{
                let originalColor = cells[i].style.backgroundColor;
                let r, g, b;
                if (originalColor === '') {{
                    r = 255;
                    g = 255;
                    b = 255;
                }} else {{
                    let rgb = originalColor.match(/rgb\\((\\d+), (\\d+), (\\d+)\\)/);
                    console.log(originalColor);
                    r = parseInt(rgb[1]);
                    g = parseInt(rgb[2]);
                    b = parseInt(rgb[3]);
                }}
                cells[i].style.backgroundColor = `rgb(${{r + 20}}, ${{g + 20}}, ${{b + 20}})`;
                
                cells[i].setAttribute('data-original-color', originalColor);
            }}
        }}
    }});
    table.addEventListener('mouseout', (event) => {{
        if (event.target.tagName === 'TD') {{
            const row = event.target.parentElement;
            row.style.backgroundColor = '';
            let cells = row.getElementsByTagName('td');
            for (let i = 0; i < cells.length; i++) {{
                let originalColor = cells[i].getAttribute('data-original-color');
                cells[i].style.backgroundColor = originalColor;
            }}
        }}
    }});
}});

document.addEventListener('DOMContentLoaded', () => {{
    const tableHeaders = document.querySelectorAll('th');
    const sortColumns = []; // Lista para manter colunas e direções de ordenação

    tableHeaders.forEach((header, index) => {{
        header.addEventListener('click', () => {{
            const table = header.closest('table');
            const tbody = table.querySelector('tbody');
            const rows = Array.from(tbody.querySelectorAll('tr'));
            const existingIndex = sortColumns.findIndex(col => col.index === index);
            
            if (existingIndex > -1) {{
                // Inverte a direção se a coluna já foi clicada antes
                sortColumns[existingIndex].asc = !sortColumns[existingIndex].asc;
            }} else {{
                // Adiciona a nova coluna com ordenação ascendente
                sortColumns.push({{ index, asc: true }});
            }}

            rows.sort((rowA, rowB) => {{
                // Itera sobre as colunas de ordenação para ordenação multi-coluna
                for (let col of sortColumns) {{
                    const textA = rowA.querySelectorAll('td')[col.index].textContent.trim().toLowerCase();
                    const textB = rowB.querySelectorAll('td')[col.index].textContent.trim().toLowerCase();

                    // Converte "-" para "0" e extrai números de strings (incluindo porcentagens)
                    const cellA = textA === '-' ? 0 : (textA.endsWith('%') ? parseFloat(textA.replace('%', '')) : parseFloat(textA));
                    const cellB = textB === '-' ? 0 : (textB.endsWith('%') ? parseFloat(textB.replace('%', '')) : parseFloat(textB));

                    if (!isNaN(cellA) && !isNaN(cellB)) {{
                        if (cellA !== cellB) {{
                            return col.asc ? cellA - cellB : cellB - cellA;
                        }}
                    }} else {{
                        if (textA !== textB) {{
                            return col.asc ? textA.localeCompare(textB) : textB.localeCompare(textA);
                        }}
                    }}
                }}
                return 0; // Se todas as colunas comparadas são iguais
            }});

            // Atualiza classes para indicar a direção da ordenação
            tableHeaders.forEach((hdr, idx) => {{
                hdr.classList.remove('asc', 'desc');
                const found = sortColumns.find(col => col.index === idx);
                if (found) {{
                    hdr.classList.add(found.asc ? 'asc' : 'desc');
                }}
            }});

            // Append sorted rows back to the tbody
            rows.forEach(row => tbody.appendChild(row));
        }});
    }});
}});
</script>
</html>
'''

with open('./data/crawled/1.4/weapons.html', 'w', encoding='utf-8') as f:
    f.write(html)